In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from scipy.stats import spearmanr, pearsonr
from joblib import Parallel, delayed
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import StandardScaler
import sys
from data import MyDataSet
import os

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
ds = MyDataSet(5)

19193 molecular features, 476 celltypes, 2659 drug_features
1135drugs and 476cell lines


In [3]:
fold = 4


In [4]:
ds.change_fold(fold, 'train')
train_rna = ds.current_molecular_data
train_experiments = ds.cases_train

ds.change_fold(fold, 'test')
test_rna = ds.current_molecular_data
test_experiments = ds.cases_test


In [5]:
print(train_rna.shape, test_rna.shape)
print(train_experiments.shape, test_experiments.shape)

(368, 19193) (108, 19193)
(424780, 8) (124772, 8)


In [6]:

drugs = train_experiments['DRUG'].drop_duplicates()
drugs

1          CYTARABINE
399        EPINASTINE
802       FLOXURIDINE
1259       VALRUBICIN
1717        ADAPALENE
             ...     
484576    PARDOPRUNOX
485928      RUBITECAN
486857         XL-647
487781      VERUBULIN
488246         VE-822
Name: DRUG, Length: 1135, dtype: object

In [7]:
rna_data_sanger = pd.read_csv('./results/data/sanger_RNA.csv',index_col=0) 
rna_data_sanger

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,H3C2,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1
cell_line,,,,,,,,,,,,,,,,,,,,,
ACH-000001,5.183487,0.000000,7.497612,2.107688,4.217231,0.042644,0.903038,5.722193,4.676944,3.720278,...,1.137504,0.000000,0.189034,1.794936,0.201634,0.000000,0.028569,0.000000,0.214125,0.014355
ACH-000002,0.176323,0.000000,5.702103,1.238787,3.119356,4.141596,0.163499,4.134221,4.111866,2.347666,...,1.952334,0.238787,0.150560,1.516015,0.000000,0.000000,0.028569,0.000000,0.000000,0.000000
ACH-000004,2.176323,0.000000,5.454505,2.480265,3.921246,0.887525,4.958843,3.949535,4.877253,4.829850,...,0.536053,0.505891,0.731183,2.643856,0.097611,0.000000,0.000000,0.000000,0.000000,0.000000
ACH-000006,0.097611,0.000000,6.285032,2.763412,4.192194,3.858976,3.921246,5.300124,4.835419,4.955592,...,0.831877,0.097611,0.575312,2.563158,1.007196,0.000000,0.000000,0.000000,0.000000,0.000000
ACH-000009,5.761019,0.111031,8.081190,2.241840,4.448901,0.000000,0.042644,7.130416,5.691255,4.432291,...,0.963474,0.000000,0.321928,2.319040,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-001603,3.152183,0.000000,6.033423,1.871844,2.664483,0.000000,0.014355,1.389567,4.053980,3.992768,...,0.855990,0.632268,0.464668,1.847997,1.084064,0.000000,0.000000,0.000000,0.000000,0.056584
ACH-000353,4.498251,0.000000,6.953149,2.060047,4.116864,0.124328,4.761817,6.214513,5.329124,3.463361,...,1.257011,0.782409,0.678072,2.488001,0.298658,0.014355,0.321928,0.014355,0.014355,0.000000
ACH-000725,3.049631,0.000000,6.812241,3.669027,4.338424,0.056584,0.275007,4.541019,5.300490,4.538538,...,3.364572,1.510962,0.594549,1.847997,0.042644,0.097611,0.014355,0.000000,0.097611,0.000000


In [8]:
sanger_results = pd.read_csv('./results/data/sanger_results.csv',index_col =0) 
sanger_results

,cell_line,drug,auc,auc_per_drug
0,ACH-000001,CAMPTOTHECIN,0.967187,1.219220
1,ACH-000002,CAMPTOTHECIN,0.711866,-1.322325
2,ACH-000004,CAMPTOTHECIN,0.742350,-1.018876
3,ACH-000006,CAMPTOTHECIN,0.752250,-0.920331
6,ACH-000009,CAMPTOTHECIN,0.948028,1.028506
...,...,...,...,...
137927,ACH-000876,JQ1-(+),0.900201,0.486184
137944,ACH-000902,JQ1-(+),0.595288,-2.043808
137948,ACH-000910,JQ1-(+),0.523851,-2.636547
137958,ACH-000930,JQ1-(+),0.830671,-0.090733


In [9]:
def crossval(drug, train_experiments, test_experiments, train_rna, test_rna, sanger_experiments, sanger_rna):
    train_experiments_now = train_experiments[train_experiments['DRUG']==drug]
    test_experiments_now = test_experiments[test_experiments['DRUG']==drug]

    train_data_and_labels = train_rna.merge(train_experiments_now, left_index=True, right_on = 'cell_line', how='inner')
    train_data = train_data_and_labels.iloc[:,:-8]  

    train_label = train_data_and_labels['auc_per_drug']
    test_data_and_labels = test_rna.merge(test_experiments_now, left_index=True, right_on = 'cell_line', how='inner')
    cell_line = test_data_and_labels['cell_line']

    test_data = test_data_and_labels.iloc[:,:-8]  
    test_label = test_data_and_labels['auc_per_drug']

    #scaling
    scaler = StandardScaler().fit(train_data)
    train_data = pd.DataFrame(scaler.transform(train_data), index = train_data.index, columns = train_data.columns)
    test_data = pd.DataFrame(scaler.transform(test_data), index = test_data.index, columns = test_data.columns)
    
    lasso = Lasso(alpha=0.1, max_iter=4000)    
    lasso_low = Lasso(alpha=1e-4, max_iter=4000)    
    
    ridge = Ridge(alpha=0.1)
    EN = ElasticNet(alpha=0.1)
    #linear = LinearRegression()
    svr = SVR(kernel='rbf')
    #kernel_ridge = KernelRidge(alpha=0.1)

    
    lasso.fit(X = train_data, y = train_label)
    prediction_lasso = lasso.predict(test_data)
    
    ridge.fit(X = train_data, y = train_label)
    prediction_ridge = ridge.predict(test_data)
    
    #kernel_ridge.fit(X = train_data, y = train_label)
    #prediction_kernel_ridge = kernel_ridge.predict(test_data)
    
    EN.fit(X = train_data, y = train_label)
    prediction_EN = EN.predict(test_data)
    
    svr.fit(X = train_data, y = train_label)
    prediction_svr = svr.predict(test_data)
    
    prism_results = pd.DataFrame({'label': test_label, 
                         'lasso': prediction_lasso, 

                         'ridge': prediction_ridge, 
                         #'kernel_ridge': prediction_kernel_ridge, 

                         'elastic': prediction_EN, 
                         'svr': prediction_svr, 
                         'drugs':drug, 
                         'fold': fold, 
                         'cells': cell_line})
    
    ############################################################
    #sanger
    sanger_experiments_now = sanger_experiments[sanger_experiments['drug']==drug]
    if sanger_experiments_now.shape[0]==0:
        return prism_results, None
    
    sanger_data_and_labels = sanger_rna.merge(sanger_experiments_now, left_index = True, right_on = 'cell_line')
    
    sanger_data = sanger_data_and_labels.iloc[:,:-4]

    sanger_data = pd.DataFrame(scaler.transform(sanger_data), index = sanger_data.index, columns = sanger_data.columns)

    sanger_label = sanger_data_and_labels['auc_per_drug']
    sanger_cell_line = sanger_data_and_labels['cell_line']
    
    sanger_results = pd.DataFrame({
        'label': sanger_label,
        'lasso': lasso.predict(sanger_data),
        'ridge': ridge.predict(sanger_data),
        'elastic': EN.predict(sanger_data),
         'svr': svr.predict(sanger_data), 
         'drugs':drug, 
         'fold': fold, 
         'cells': sanger_cell_line  
    })
    
    ############################################################
    print(train_data.shape, test_data.shape, sanger_data.shape)
    
    return prism_results, sanger_results


In [10]:
crossval(np.array(drugs)[0], train_experiments=train_experiments, test_experiments=test_experiments, train_rna=train_rna, test_rna=test_rna, sanger_experiments = sanger_results, sanger_rna = rna_data_sanger)

(303, 19193) (95, 19193) (235, 19193)


(        label     lasso     ridge   elastic       svr       drugs  fold   
 0    2.367402  0.210130 -0.179300 -0.128321 -0.106837  CYTARABINE     4  \
 9    0.498390  0.007921 -0.248373 -0.122444  0.083703  CYTARABINE     4   
 11  -0.989694 -0.269812  0.389163 -0.448085 -0.069980  CYTARABINE     4   
 14   0.807546  0.107736  0.208689  0.625824 -0.097260  CYTARABINE     4   
 15   1.642329 -0.194511 -0.697427 -0.150100 -0.073506  CYTARABINE     4   
 ..        ...       ...       ...       ...       ...         ...   ...   
 362 -1.083521 -0.206357 -0.846628 -0.447085 -0.184358  CYTARABINE     4   
 367 -1.249794 -0.052251 -0.033104 -0.282618 -0.011757  CYTARABINE     4   
 369  1.439482 -0.250933  0.394553 -0.152805  0.188753  CYTARABINE     4   
 375  2.166002  0.122337  0.797879  0.797555 -0.256837  CYTARABINE     4   
 387  0.649756  0.211220 -0.296067  0.053956  0.057600  CYTARABINE     4   
 
           cells  
 0    ACH-000879  
 9    ACH-000832  
 11   ACH-000137  
 14   ACH-

In [11]:
#all_tests = [crossval(drug, train_experiments=train_experiments, test_experiments=test_experiments, train_rna=train_rna, test_rna=test_rna ) for drug in tqdm(np.array(drugs))]

In [12]:
all_tests = Parallel(n_jobs=10)(delayed(crossval)(drug,train_experiments, test_experiments, train_rna, test_rna, sanger_experiments = sanger_results, sanger_rna = rna_data_sanger) for drug in tqdm(np.array(drugs)[:]))


  1%|▎                                        | 10/1135 [00:00<00:38, 29.48it/s]/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy vers

  7%|██▉                                      | 80/1135 [00:46<12:39,  1.39it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
  8%|███▎                                     | 90/1135 [00:51<11:22,  1.53it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered 

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret 

 31%|████████████▎                           | 350/1135 [03:44<07:25,  1.76it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 32%|████████████▋                           | 360/1135 [03:50<07:39,  1.69it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered 

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 39%|███████████████▌                        | 440/1135 [04:52<08:59,  1.29it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

...

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 72%|████████████████████████████▉           | 820/1135 [09:17<03:56,  1.33it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 78%|███████████████████████████████▎        | 890/1135 [09:57<02:20,  1.74it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

 85%|██████████████████████████████████▏     | 970/1135 [10:41<01:31,  1.80it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 86%|██████████████████████████████████▌     | 980/1135 [10:45<01:20,  1.92it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 87%|██████████████████████████████████▉     | 990/1135 [10:54<01:30,  1.59it/s]/home/pkeyl/.local/lib/python3.10/site-p

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 93%|████████████████████████████████████   | 1050/1135 [11:37<00:57,  1.48it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 98%|██████████████████████████████████████▏| 1110/1135 [12:21<00:19,  1.31it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

In [13]:
all_tests_combined_prism = pd.concat([tup[0] for tup in all_tests], axis=0)
all_tests_combined_sanger = pd.concat([tup[1] for tup in all_tests], axis=0)

all_tests_combined_prism

,label,lasso,ridge,elastic,svr,drugs,fold,cells
0,2.367402,0.210130,-0.179300,-0.128321,-0.106837,CYTARABINE,4,ACH-000879
9,0.498390,0.007921,-0.248373,-0.122444,0.083703,CYTARABINE,4,ACH-000832
11,-0.989694,-0.269812,0.389163,-0.448085,-0.069980,CYTARABINE,4,ACH-000137
14,0.807546,0.107736,0.208689,0.625824,-0.097260,CYTARABINE,4,ACH-000630
15,1.642329,-0.194511,-0.697427,-0.150100,-0.073506,CYTARABINE,4,ACH-000266
...,...,...,...,...,...,...,...,...
488690,-0.137179,0.633408,0.069416,0.804824,0.362443,VE-822,4,ACH-000791
488700,-0.262404,-0.310092,-0.405998,-0.157484,-0.077951,VE-822,4,ACH-000252
488493,-0.174391,0.010803,0.932002,2.200632,0.149575,VE-822,4,ACH-000375
488342,0.432468,0.043196,0.355934,-0.065649,0.206965,VE-822,4,ACH-000945


In [14]:
if not os.path.exists('./results/other_models/'):
    os.makedirs('./results/other_models')

all_tests_combined_sanger.to_csv('./results/other_models/sanger'+str(fold)+'.csv')
all_tests_combined_prism.to_csv('./results/other_models/prism'+str(fold)+'.csv')


(705, 19193) (201, 19193) (229, 19193)
(347, 19193) (104, 19193) (267, 19193)
(337, 19193) (94, 19193) (156, 19193)
(347, 19193) (96, 19193) (238, 19193)
(292, 19193) (90, 19193) (408, 19193)
(356, 19193) (106, 19193) (257, 19193)
(675, 19193) (196, 19193) (246, 19193)
(320, 19193) (97, 19193) (255, 19193)
(235, 19193) (71, 19193) (267, 19193)
(331, 19193) (98, 19193) (245, 19193)
(362, 19193) (103, 19193) (253, 19193)
(359, 19193) (107, 19193) (167, 19193)
(355, 19193) (107, 19193) (255, 19193)
(340, 19193) (104, 19193) (23, 19193)
(637, 19193) (177, 19193) (238, 19193)
(696, 19193) (205, 19193) (262, 19193)
(610, 19193) (179, 19193) (262, 19193)
(356, 19193) (103, 19193) (263, 19193)
(356, 19193) (102, 19193) (264, 19193)
(361, 19193) (105, 19193) (268, 19193)
(693, 19193) (196, 19193) (169, 19193)
(356, 19193) (106, 19193) (275, 19193)
(357, 19193) (107, 19193) (17, 19193)
(322, 19193) (97, 19193) (262, 19193)
(360, 19193) (104, 19193) (233, 19193)
(350, 19193) (102, 19193) (236, 19